<a href="https://colab.research.google.com/github/araya1203/study_data_analytics/blob/main/codes/NLP/healthapp_review_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# refer : https://heytech.tistory.com/401

In [2]:
# !pip install python-mecab-ko pyLDAvis

In [3]:
import pandas as pd

df_review_negative = pd.read_csv('./review_negative.csv')


In [4]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        413 non-null    int64 
 1   app               413 non-null    object
 2   review            413 non-null    object
 3   rating            413 non-null    int64 
 4   tokenized_review  346 non-null    object
 5   replaced_review   346 non-null    object
dtypes: int64(2), object(4)
memory usage: 19.5+ KB


In [5]:
df_review_negative = df_review_negative.dropna()

### LDA 분석

#### 문장 벡터화

In [6]:
!pip install --upgrade numpy

In [7]:
from gensim.corpora import Dictionary

In [8]:
df_review_negative[['replaced_review']]

,replaced_review
3,칼로리
5,마음
7,오류
8,모야
9,느림
...,...
406,운동 리뷰 출창
407,광고 짜증
408,운동 친절 운동 운동 체계 순서 운동 기분 시작 운동 구간 시작
409,광고 운동


In [9]:
dictionary = Dictionary(df_review_negative[['replaced_review']].values)


In [10]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

[(3, 1)]

In [13]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [14]:
df_review_negative['문장벡터화'] = corpus_list

In [15]:
df_review_negative.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장벡터화'],
      dtype='object')

### 토픽 잡기

In [16]:
from gensim.models import LdaModel

In [17]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=3)

In [18]:
lda_model.print_topics(num_words=4)

[(0, '0.025*"운동" + 0.014*"결제 취소" + 0.010*"리뷰" + 0.009*"결제"'),
 (1, '0.026*"환불" + 0.026*"로그인" + 0.013*"결제 환불" + 0.009*"자전거 인식 걸음"'),
 (2, '0.014*"불편" + 0.010*"광고" + 0.010*"최악" + 0.010*"자동 결제 취소"')]

#### quest
- review_positive.csv 토픽 분석
- 토픽 수는 3으로 가정

In [34]:
df_review_positive = pd.read_csv('./review_positive.csv')

In [35]:
df_review_positive = df_review_positive.dropna()

In [36]:
df_review_positive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4351 entries, 1 to 7585
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        4351 non-null   int64 
 1   app               4351 non-null   object
 2   review            4351 non-null   object
 3   rating            4351 non-null   int64 
 4   tokenized_review  4351 non-null   object
 5   replaced_review   4351 non-null   object
dtypes: int64(2), object(4)
memory usage: 237.9+ KB


In [37]:
df_review_positive[['replaced_review']]

,replaced_review
1,최고
4,필요 운동 최고
5,운동 최고
7,집중 운동
12,운동 도움
...,...
7579,리뷰 설명 타임 운동 추천 건강
7580,운동 몸무게 점차 저녁 기대
7582,추천
7583,운동 영상


In [38]:
dictionary_positive = Dictionary(df_review_positive[['replaced_review']].values)

In [39]:
dictionary_positive.doc2bow(df_review_positive[['replaced_review']].values[3])

[(3, 1)]

In [40]:
# dictionary_positive에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary_positive.doc2bow(sentence)
  corpus_list.append(vectors)

In [41]:
df_review_positive['문장벡터화'] = corpus_list

In [42]:
df_review_positive.columns

Index(['Unnamed: 0', 'app', 'review', 'rating', 'tokenized_review',
       'replaced_review', '문장벡터화'],
      dtype='object')

In [43]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary_positive, num_topics=3)

In [44]:
lda_model.print_topics(num_words=4)

[(0, '0.275*"운동" + 0.034*"추천" + 0.015*"운동 도움" + 0.012*"도전"'),
 (1, '0.039*"뱃살" + 0.029*"감사" + 0.025*"도움" + 0.015*"강추"'),
 (2, '0.082*"최고" + 0.017*"광고" + 0.014*"운동 최고" + 0.011*"유용"')]

### LDA 시각화

In [45]:
%pip install pyLDAvis

In [51]:
import pyLDAvis.gensim_models

In [52]:
pyLDAvis.enable_notebook()
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [53]:
pyLDAvis.display(result_visualized)

In [54]:
pyLDAvis.save_html(result_visualized, './result_visualized.html')